# DNN

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import h5py

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or '3' to suppress most of the logs
tf.get_logger().setLevel('WARNING')  # Adjust logging level

2023-12-27 14:33:13.754929: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-27 14:33:15.432343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 14:33:15.432378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 14:33:15.625560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 14:33:16.014207: I tensorflow/core/platform/cpu_feature_guar

In [2]:
X = np.load('images64.npy')
y = np.load('styles64.npy')
# X and y are now numpy arrays containing your data

In [3]:
np.unique(y)

array(['Classic', 'Contemporary', 'Country', 'Minimalism', 'Modern',
       'Unique', 'Urban'], dtype='<U12')

In [4]:
from sklearn.calibration import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)



from tensorflow.keras.utils import to_categorical

# Assuming y contains integer labels
y = to_categorical(y, num_classes=7)

In [5]:
from sklearn.model_selection import train_test_split
 

# Now proceed with the train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

In [6]:
print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(9400, 64, 64, 3) (1175, 64, 64, 3) (1175, 64, 64, 3)
(9400, 7) (1175, 7) (1175, 7)


In [7]:
X_train.shape[1:]

(64, 64, 3)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Dropout, Conv2D, AvgPool2D, Flatten, LeakyReLU

In [9]:
model = Sequential([
    Input(X_train.shape[1:]),
    
    Conv2D(filters=32, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Conv2D(filters=16, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Conv2D(filters=8, kernel_size=(4,4), strides=(1,1), padding='same'),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    AvgPool2D(pool_size=(4,4)),
    
    Flatten(),
    
    Dense(128),
    BatchNormalization(),
    LeakyReLU(alpha=0.01),
    Dropout(0.2),
    
    Dense(7),
    Activation('softmax')
])

model.summary()

2023-12-27 14:33:46.289932: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-27 14:33:47.608158: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-27 14:33:47.608277: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        1568      
                                                                 
 batch_normalization (Batch  (None, 64, 64, 32)        128       
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64, 64, 32)        0         
                                                                 
 average_pooling2d (Average  (None, 16, 16, 32)        0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 16)        8208      
                                                                 
 batch_normalization_1 (Bat  (None, 16, 16, 16)        6

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               verbose=1,
                               restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=5,
                              verbose=1,
                              min_lr=0.001)


In [11]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print('start fitting')
history = model.fit(
    X_train, y_train,
    epochs=10000000,
    batch_size=32,
    validation_data=(X_val,y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
)

print('fitting done')

Num GPUs Available:  1
start fitting
Epoch 1/10000000


2023-12-27 14:33:50.988835: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-27 14:33:54.961023: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa4d1b97cd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-27 14:33:54.961038: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2023-12-27 14:33:55.106995: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703655235.272523   93079 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


294/294 [==============================] - 9s 6ms/step - loss: 1.1444 - accuracy: 0.6666 - val_loss: 1.1394 - val_accuracy: 0.6970 - lr: 0.0010
Epoch 2/10000000
294/294 [==============================] - 1s 3ms/step - loss: 1.0073 - accuracy: 0.7010 - val_loss: 1.4097 - val_accuracy: 0.5421 - lr: 0.0010
Epoch 3/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.9606 - accuracy: 0.7078 - val_loss: 0.9774 - val_accuracy: 0.7098 - lr: 0.0010
Epoch 4/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.9338 - accuracy: 0.7139 - val_loss: 1.3121 - val_accuracy: 0.6851 - lr: 0.0010
Epoch 5/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.8965 - accuracy: 0.7248 - val_loss: 1.0946 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 6/10000000
294/294 [==============================] - 1s 3ms/step - loss: 0.8775 - accuracy: 0.7339 - val_loss: 1.1785 - val_accuracy: 0.6468 - lr: 0.0010
Epoch 7/10000000
294/294 [=========================

In [12]:
y_pred = model.predict(X_test)
print(y_pred[0])

import numpy as np

y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels[:10]



37/37 [==============================] - 0s 1ms/step
[1.0960133e-02 1.0858343e-02 1.2964699e-01 9.6850577e-05 7.1801579e-01
 4.8754831e-05 1.3037305e-01]


array([4, 4, 4, 4, 2, 4, 2, 4, 4, 4])

In [13]:
if y_test.ndim > 1:  # Check if y_test is one-hot encoded
    y_test_int_labels = np.argmax(y_test, axis=1)
else:
    y_test_int_labels = y_test  # y_test is already in integer label format

# Now calculate the accuracy
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test_int_labels, y_pred_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test_int_labels, y_pred_labels))
print("Classification Report:\n", classification_report(y_test_int_labels, y_pred_labels))


Accuracy: 0.7838297872340425
Confusion Matrix:
 [[ 54   9   4   0  60   0   1]
 [  3  22   5   1  56   0   0]
 [  4   4  43   0  22   0   0]
 [  1   0   0   4  11   0   0]
 [ 15  12  14   0 768   0   9]
 [  0   0   0   0   4   3   0]
 [  0   2   1   0  16   0  27]]
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.42      0.53       128
           1       0.45      0.25      0.32        87
           2       0.64      0.59      0.61        73
           3       0.80      0.25      0.38        16
           4       0.82      0.94      0.88       818
           5       1.00      0.43      0.60         7
           6       0.73      0.59      0.65        46

    accuracy                           0.78      1175
   macro avg       0.73      0.50      0.57      1175
weighted avg       0.77      0.78      0.76      1175



# ML

In [14]:
X = np.load('images64.npy')
y = np.load('styles64.npy')
# X and y are now numpy arrays containing your data

In [15]:
from sklearn.calibration import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

# Reshape X from (len(X), 64, 64, 3) to (len(X), 1, 12288)
X = X.reshape(len(X),-1)  # -1 tells numpy to infer the dimension

from sklearn.model_selection import train_test_split

# Now proceed with the train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print(X_train.shape, X_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

(9400, 12288) (1175, 12288) (1175, 12288)
(9400,) (1175,) (1175,)


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [17]:
# Random Forest
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)

# Predictions
y_pred_RFC = RFC.predict(X_test)

# Accuracy
accuracy_RFC = accuracy_score(y_test, y_pred_RFC)
print(f"Accuracy: {accuracy_RFC * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_RFC = precision_score(y_test, y_pred_RFC, average='weighted')
recall_RFC = recall_score(y_test, y_pred_RFC, average='weighted')
f1_RFC = f1_score(y_test, y_pred_RFC, average='weighted')

print(f"Precision: {precision_RFC:.2f}")
print(f"Recall: {recall_RFC:.2f}")
print(f"F1 Score: {f1_RFC:.2f}")

# Confusion Matrix
conf_matrix_RFC = confusion_matrix(y_test, y_pred_RFC)
print("Confusion Matrix:")
print(conf_matrix_RFC)


Accuracy: 87.91%
Precision: 0.89
Recall: 0.88
F1 Score: 0.87
Confusion Matrix:
[[ 72   0   0   0  56   0   0]
 [  0  47   0   0  40   0   0]
 [  1   1  53   0  19   0   0]
 [  1   0   0  12   3   0   0]
 [  5   0   0   0 814   0   0]
 [  0   0   0   0   3   3   0]
 [  0   0   0   0  13   0  32]]


In [18]:
# SVM Classifier
SVM = SVC()

# Random Forest
SVM = RandomForestClassifier()
SVM.fit(X_train, y_train)

# Predictions
y_pred_SVM = SVM.predict(X_test)

# Accuracy
accuracy_SVM = accuracy_score(y_test, y_pred_SVM)
print(f"Accuracy: {accuracy_SVM * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_SVM = precision_score(y_test, y_pred_SVM, average='weighted')
recall_SVM = recall_score(y_test, y_pred_SVM, average='weighted')
f1_SVM = f1_score(y_test, y_pred_SVM, average='weighted')

print(f"Precision: {precision_SVM:.2f}")
print(f"Recall: {recall_SVM:.2f}")
print(f"F1 Score: {f1_SVM:.2f}")

# Confusion Matrix
conf_matrix_SVM = confusion_matrix(y_test, y_pred_SVM)
print("Confusion Matrix:")
print(conf_matrix_SVM)



Accuracy: 88.26%
Precision: 0.90
Recall: 0.88
F1 Score: 0.87
Confusion Matrix:
[[ 74   0   0   0  54   0   0]
 [  0  47   0   0  40   0   0]
 [  1   0  53   0  20   0   0]
 [  0   0   0  12   4   0   0]
 [  3   0   0   0 816   0   0]
 [  0   0   0   0   3   3   0]
 [  0   0   0   0  13   0  32]]


In [19]:
import joblib

# Assuming your SVM model is named as SVM
joblib.dump(SVM, 'svm_model.joblib')

# # Load the model from the file
# SVM_model = joblib.load('svm_model.joblib')

['svm_model.joblib']

In [20]:
# XGBClassifier
XGB= xgb.XGBClassifier()

XGB.fit(X_train, y_train)

# Predictions
y_pred_XGB = XGB.predict(X_test)

# Accuracy
accuracy_XGB = accuracy_score(y_test, y_pred_XGB)
print(f"Accuracy: {accuracy_XGB * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_XGB = precision_score(y_test, y_pred_XGB, average='weighted')
recall_XGB = recall_score(y_test, y_pred_XGB, average='weighted')
f1_XGB = f1_score(y_test, y_pred_XGB, average='weighted')

print(f"Precision: {precision_XGB:.2f}")
print(f"Recall: {recall_XGB:.2f}")
print(f"F1 Score: {f1_XGB:.2f}")

# Confusion Matrix
conf_matrix_XGB = confusion_matrix(y_test, y_pred_XGB)
print("Confusion Matrix:")
print(conf_matrix_XGB)



Accuracy: 87.83%
Precision: 0.89
Recall: 0.88
F1 Score: 0.87
Confusion Matrix:
[[ 75   0   0   0  53   0   0]
 [  0  47   0   1  39   0   0]
 [  1   0  53   0  19   0   1]
 [  1   0   0  12   3   0   0]
 [  7   1   1   0 810   0   0]
 [  0   0   0   0   3   3   0]
 [  0   0   0   0  13   0  32]]


In [21]:
# Naive Bayes
GNB= GaussianNB()

GNB.fit(X_train, y_train)

# Predictions
y_pred_GNB = GNB.predict(X_test)

# Accuracy
accuracy_GNB = accuracy_score(y_test, y_pred_GNB)
print(f"Accuracy: {accuracy_GNB * 100:.2f}%")

# Precision, Recall, and F1 Score
precision_GNB = precision_score(y_test, y_pred_GNB, average='weighted')
recall_GNB = recall_score(y_test, y_pred_GNB, average='weighted')
f1_GNB = f1_score(y_test, y_pred_GNB, average='weighted')

print(f"Precision: {precision_GNB:.2f}")
print(f"Recall: {recall_GNB:.2f}")
print(f"F1 Score: {f1_GNB:.2f}")

# Confusion Matrix
conf_matrix_GNB = confusion_matrix(y_test, y_pred_GNB)
print("Confusion Matrix:")
print(conf_matrix_GNB)



Accuracy: 27.40%
Precision: 0.62
Recall: 0.27
F1 Score: 0.34
Confusion Matrix:
[[ 22  23   4  10  19  11  39]
 [ 13  21   1  12  18   9  13]
 [  7  16  20   3   9   3  16]
 [  1   3   0   3   0   1   8]
 [ 71  98  51 103 223 108 165]
 [  0   0   0   0   0   3   3]
 [  5   2   1   2   4   1  30]]
